# A notebook to reproduce a 3D visualization of cell nuclei for 4DN

# Import libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import cm
import napari
from scipy.spatial import ConvexHull

# Load data

In [2]:
# Define paths
fl_dna, fl_protein, fl_rna = ['data/7_27_2019__Conv_zscan_09__cell9_DNA.csv', # was _Protein in original code for some reason
                              'data/7_27_2019__Conv_zscan_09__cell9_Protein.csv', 
                              'data/7_27_2019__Conv_zscan_09__cell9_RNA.csv']

# Load datasets as DataFrames
df_dna = pd.read_csv(fl_dna)
df_prot = pd.read_csv(fl_protein)
df_rna = pd.read_csv(fl_rna)

# Preprocess data

In [3]:
# Get 3D positions for chromosomes
X = np.array(df_dna[['z','x','y']])

# Preprocess chromosomes
chrs = [nm.split(':')[0] for nm in df_dna['genomic_coordinate']]
ichrs = np.arange(23)
ichrs_ = ichrs/np.max(ichrs)


cols_chr = {'chr'+(str(i+1) if i<=21 else 'X'):cm.rainbow(ichrs_[i]) for i in ichrs}
cols = np.array([cols_chr[chr] for chr in chrs])
keep = ~np.any(np.isnan(X),axis=-1)


# Intialize Napari viewer
V = napari.Viewer()

# Add DNA as points
V.add_points(X[keep],size=0.2,face_color=cols[keep],shading='spherical',name='DNA')

 
# Add proteins
XP = np.array(df_prot[['z','x','y']])
keepP = df_prot['protein']=='nucleoli'

# Add proteins to viewer
V.add_points(XP[keepP],size=0.1,face_color='cyan',edge_width=0,shading='spherical',name = 'nucleoli')
keepP = df_prot['protein']=='speckles'
V.add_points(XP[keepP],size=0.1,face_color='yellow',edge_width=0,shading='spherical',name = 'speckles')

# Get convex hull
hull = ConvexHull(X[keep])
hulls = hull.points[hull.simplices]
V.add_shapes(hulls, shape_type='polygon',edge_width=0.1, edge_color='gray', face_color='gray', name='lamina')

# Add 3D positions for RNA
XR = np.array(df_rna[['z','x','y']])
bR = df_rna['brightness']
vmax = np.percentile(bR,99)
bRn = np.clip(bR/vmax,0,1)
ugenes = list(np.unique(df_rna['gene']))
gns = list(df_rna['gene'])

# Define and apply colors
colsRu = ["#e6194B", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", "#42d4f4", "#f032e6", "#bfef45",

        "#fabed4", "#469990", "#dcbeff", "#9A6324", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1",

        "#000075", "#a9a9a9"]
colsR = [colsRu[gns.index(gn)%len(colsRu)] for gn in gns]

# Add 3D oints for nascent RNA
V.add_points(XR,size=0.4*bRn+0.1,face_color=colsR,edge_width=0,shading='spherical',name='nascentRNA')

/var/folders/x7/ttglg94d4g500ypbdkw3__nw0000gq/T/ipykernel_42906/2231830169.py:27: FutureWarning: Argument 'edge_width' is deprecated, please use 'border_width' instead. The argument 'edge_width' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  V.add_points(XP[keepP],size=0.1,face_color='cyan',edge_width=0,shading='spherical',name = 'nucleoli')
/var/folders/x7/ttglg94d4g500ypbdkw3__nw0000gq/T/ipykernel_42906/2231830169.py:29: FutureWarning: Argument 'edge_width' is deprecated, please use 'border_width' instead. The argument 'edge_width' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  V.add_points(XP[keepP],size=0.1,face_color='yellow',edge_width=0,shading='spherical',name = 'speckles')
/var/folders/x7/ttglg94d4g500ypbdkw3__nw0000gq/T/ipykernel_42906/2231830169.py:53: FutureWarning: Argument 'edge_width' is deprecated, please use 'border_width' instead. The argument 'edge_width' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  V.add_points(XR,size=0.4*bRn+

<Points layer 'nascentRNA' at 0x341f7d7c0>

# Run Napari

In [4]:
# Start Napari event loop
napari.run()

2024-08-01 10:37:50.583 Python[42906:20094184] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
